# Training and Modelling for the TS based Cov-Matrix Picker

In [76]:
import torch, os
import pandas as pd
import numpy as np
import pickle
from numpy import random as npr
import torch.nn as nn

## Support Functions

In [77]:
def which_mat(TTS, choice, n = [7, 7*2, 7*4, 7*4*2, 7*4*3], p = [0], channel_use = 1):
    """
    Funciton which can be easilly calle din order to implament which_mat as part of a larger function
    """
    return get_mat(TTS, n[choice[0]], p[choice[1]], channel_use)

def get_mat(TTS, n,p = 0, channel_use = 1):
    """
    Funciton that takes in 
    TTS Tesnor of Time Serieses, and given 
    n number of period to use and
    p  last index period to use, 
    return a covariance matrix of the data for that period in terms of the values in chanel channel_use
    """
    
    d1, d2 = TTS[0], TTS[1]
    #insert function to partition TTS correctly
    
    #creating cova matrix estimation desired
    d = [d1, d1]
    return pd.DataFrame({"S%d" %i: d for i,d in  enumerate(d)}).cov()


def cov_matrix_loss(A,B, type_use = 1):
    """
    Get distance between matracies based either on component wise distance or using eigenvalues
    """
    if type_use == 0:
        ind = np.triu_indices(2)
        loss = nn.MSELoss()
        return loss(A[ind], B[ind]) 
    
    elif type_use == 1:
        return torch.sqrt(torch.sum(torch.pow(torch.log(torch.eig(A-B))),2))
    
    

## NN Models

In [125]:
class TS_based_simple(nn.Module):
    def __init__(self, num_outs,
                 kernel = 5, in_channels = 4,  hidden_size = 128):
        """
        
        
        """
        super(TS_based_simple, self).__init__()
        
        pads = kernel // 2 
        
        self.downconv1 = nn.Sequential(
            nn.Conv1d(in_channels = in_channels, out_channels = hidden_size, 
                      stride = 2,kernel_size = kernel, padding = pads ),
            nn.BatchNorm1d(hidden_size),
            nn.ReLU(),
            nn.MaxPool1d(2)
            )
        self.downconv2 = nn.Sequential(
            nn.Conv1d(in_channels = hidden_size, out_channels = (hidden_size/2), 
                      kernel_size = (kernel-2), stride = 2, padding = (pads-1) ),
            nn.BatchNorm1d(hidden_size/2),
            nn.ReLU(),
            nn.MaxPool1d(2)
            )
        
        self.downconv3 = nn.Sequential(
            nn.Conv1d(in_channels = (hidden_size/2), out_channels = (hidden_size/(2**2)), 
                      kernel_size = (kernel-2), stride = 2, padding = (pads-1) ),
            nn.BatchNorm1d(hidden_size/(2**2)),
            nn.ReLU(),
            nn.MaxPool1d(2)
            )
        
        self.downconvLast = nn.Sequential(
            nn.Conv1d(in_channels = (hidden_size/(2**2)) , out_channels = num_outs, 
                      kernel_size = (kernel-2), padding = (pads-1)),
            nn.BatchNorm1d(),
            nn.ReLU(),
            nn.MaxPool1d()
            )
        
        self.picker = nn.Linear(out_channels = num_outs)
        
    
    def forward(self, input):
        
        input_1, input_2 = input[0], input[1]
        
        #run the model on the first TS
        self.c11 = self.cov1(input_1)
        self.c12 = self.cov2(self.c11)
        self.c13 = self.cov3(self.c12)
        
        #run the model on the second TS
        self.c21 = self.cov1(input_2)
        self.c22 = self.cov2(self.c11)
        self.c23 = self.cov3(self.c12)
        
        #concat the processed data and downconvthat
        self.cl = self.downconvLast(torch.cat((self.c23, self.c3), dim=1))
        
        #concat TS based data to data representative of other info
        self.last = self.picker(self.cl)
        

class TS_based_wfactors(nn.Module):
    def __init__(self, num_outs, kernel = 5, channels_in =4, hidden_size = 128):
        """
        
        
        """
        super(TS_based_wfactors, self).__init__()
        
        pads  == kernel//2
        
        self.downconv1 = nn.Sequential(
            nn.Conv1d(in_channels = channels_in, out_channels = hidden_size, 
                      stride = 2,kernel_size = kernel, padding = pads ),
            nn.BatchNorm1d(hidden_size),
            nn.ReLU(),
            nn.MaxPool1d(2)
            )
        self.downconv2 = nn.Sequential(
            nn.Conv1d(in_channels = hidden_size, out_channels = (hidden_size/2), 
                      kernel_size = (kernel-2), stride = 2, padding = (pads-1) ),
            nn.BatchNorm1d(hidden_size/2),
            nn.ReLU(),
            nn.MaxPool1d(2)
            )
        
        self.downconv3 = nn.Sequential(
            nn.Conv1d(in_channels = (hidden_size/2), out_channels = (hidden_size/(2**2)), 
                      kernel_size = (kernel-2), stride = 2, padding = (pads-1)),
            nn.BatchNorm1d(hidden_size/(2**2)),
            nn.ReLU(),
            nn.MaxPool1d(2)
            )
        
        self.downconvLast = nn.Sequential(
            nn.Conv1d(in_channels = (hidden_size/(2**2)) , out_channels = num_outs, 
                      kernel_size = (kernel-2), padding = (pads-1)),
            nn.BatchNorm1d(),
            nn.ReLU(),
            nn.MaxPool1d()
            )
        
        self.picker = nn.Linear(in_channels = num_outs , out_channels = num_outs)
        
    
    def forward(self, input):
        
        input_1, input_2 = input[0], input[1]
        
        #run the model on the first TS
        self.c11 = self.cov1(input_1)
        self.c12 = self.cov2(self.c11)
        self.c13 = self.cov3(self.c12)
        
        #run the model on the second TS
        self.c21 = self.cov1(input_2)
        self.c22 = self.cov2(self.c11)
        self.c23 = self.cov3(self.c12)
        
        #concat the processed data and downconvthat
        self.cl = self.downconvLast(torch.cat((self.c23, self.c3), dim=1))
        
        #concat TS based data to data representative of other info
        self.last = self.picker(self.cl)
        

## Trainer

In [126]:
def training_set(data_use, max_num_training):
    pop

def train(train_d, args, mod_use = None):
    #setting up the packages
    torch.set_num_threads(5)
    npr.seed(args.seed)
    dir_save_to = "results/" + args.dir_name
    if not os.path.exists(dir_save_to):
        os.makedirs(dir_save_to)
    
    
    #type of metric to use for loss:
    l_t = 1
    if arg.loss_type == "L2":
        l_t = 0
    
    
    #which model to implament
    if mod_use is None:
        if args.model_use == "Mod1":
              mod_use = TS_based_simple(num_outs = args.k_size) #args.num_filters, num_colours, arg.num_in_chans)
        elif args.model_use == "Mod1wFactors":
              mod_use = TS_based__wfactors(num_outs = args.k_size) #, args.num_filters, num_colours, arg.num_in_chans)
    
    #setting up the model's optimizaer
    optimizer = torch.optim.Adam(mod_use.parameters(), lr=args.lrn_rate)
    
    #where we are going to gather data
    hist_tr_loss = []
    hist_tt_loss = []
    
    #let's start training
    for i in range(arg.num_epochs):
        epoch_loss = []
        
        for j in range(len(training_set(train_d, a))):
            #setup optimizer
            optimizer.zero_grad()
            
            #set up optimization
            output_choice = mod_use(input_d)
            CM_use = which_mat(output_choice)
            lossing_me = cov_matrix_loss(output_chice, CM_use, l_t)
            
            #optimizing
            lossing.backward()
            optimizer.step()
            epoch_loss.append(lossing.data.item())
            
        
        #printing some stuff
        avg_loss = np.mean(losses)
        train_losses.append(avg_loss)

In [127]:
#setting up args
class AttrDict(dict):
    def __init__(self, *args, **kwargs):
        super(AttrDict, self).__init__(*args, **kwargs)
        self.__dict__ = self

arg = AttrDict()
args = {"seed": 12345,
        "GPU" : "TRUE",
        "dir_name" : "trial1",
        "lrn_rate" : 0.005,
        "model_use" : "Mod1",
        "num_in_chans" : 4,
        "k_size" : 5,
        "loss_type" : "L2",
        "num_epochs" : 10,
        "max_train" : 2*10**4,
        "num_filters" : 128
}
arg.update(args)
train_d = "pickle_that_pickles" #where are the pickleessss

In [128]:
#model
the_model = train(train_d, arg)

TypeError: new() received an invalid combination of arguments - got (float, int, int), but expected one of:
 * (torch.device device)
 * (torch.Storage storage)
 * (Tensor other)
 * (tuple of ints size, torch.device device)
 * (object data, torch.device device)


In [74]:
sp500 = open("../Data Sets/Constituent Data/SP500.pickle", 'r')
pickle.load(sp500)

FileNotFoundError: [Errno 2] No such file or directory: '../Data Sets/Constituent Data/SP500.pickle'

In [ ]:
"../Data Sets/"